<a href="https://colab.research.google.com/github/AlexanderPushkarev/LangChain/blob/main/%22%D0%91%D0%94_%D0%BD%D0%B0_FAISS%20%D0%B8%20%D0%BF%D0%BE%D0%B8%D1%81%D0%BA%D0%BE%D0%B2%D0%B8%D0%BA%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загрузка библиотек и работа с базой данных

In [2]:
#Устанавливаем библиотеки
!pip install -q langchain langchain-community sentence_transformers faiss-cpu
#если есть возможность подключить видеокарту,  faiss-cpu меняем на faiss-gpu
#Чтобы работало без сбоев, можно зафиксировать версии, например langchain==0.3.18 sentence_transformers==3.4.1 faiss-cpu==1.10.0 langchain-community==0.3.18

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.

In [3]:
#!pip show langchain-community
#посмотреть версию какой-то из библиотек

In [4]:
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_community. vectorstores import FAISS
from langchain.document_loaders import TextLoader
import re
import pandas as pd

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name='cointegrated/rubert-tiny2',
                                        #multi_process=True, model_kwargs={"device": "cuda"}, раскомментировать если работаем с GPU
                                         encode_kwargs={"normalize_embeddings": True})
# есть работаем без виде карты удаляем или закомментируем кусок кода выше model_kwargs={"device": "cuda"}, иначе если не найдет видеокарту, выдаст ошибку. Без видеокарты будет работать медленнее

<ipython-input-5-fced627d6c7d>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='cointegrated/rubert-tiny2',
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
#Тестовая база на 20 ГБ
#! wget 'https://opendata.trudvsem.ru/csv/cv.csv'

In [ ]:
#Если мы хотим работать с полной базой
df_resum_all = pd.read_csv('/content/cv.csv', on_bad_lines='skip', nrows=100000, sep='|')

<ipython-input-18-2f68a2749fad>:2: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_resum_all = pd.read_csv('/content/cv.csv', on_bad_lines='skip', nrows=100000, sep='|')


In [7]:
df_resum = pd.read_excel ('/content/samp.xlsx')




In [9]:
#Загружаем базу данных,
df_resum = pd.read_excel('/content/samp.xlsx')
#Самостоятельно получить подвыборку в 5000 строк
#df_resum = df_resum_all.sample(5000)

In [10]:
#Смотрим на три первые строки
df_resum.head(3)

,Unnamed: 0,id,candidateId,stateRegionCode,locality,localityName,birthday,gender,age,positionName,...,softSkills,workExperienceList,scheduleType,salary,busyType,retrainingCapability,businessTrip,languageKnowledge,relocation,innerInfo
0,33163,12c09410-6eda-11ed-8193-e57b87a63fa7,85f44450-6ed9-11ed-b741-732207d240c8,7500000000000,7502300001300,"Забайкальский-край,-Тунгокоченский-район,-Верш...",1971-05-29T00:00:00+0300,Мужской,53.0,разнорабочий,...,[],"[{""companyName"": ""АО ЗабТЭК"", ""dateFrom"": ""202...",Полный-рабочий-день,35000,Полная-занятость,Не-готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,"{""idUser"": ""f09dba60-1e6f-11e5-8442-1ff7059456..."
1,36612,12cf21e0-fc4c-11ea-9f9c-69632329477c,1f483d80-f9e0-11ea-a1ca-7bf9d8e248ac,1600000000000,1600000000000,Татарстан-республика,1988-11-23T00:00:00+0300,Мужской,35.0,Кладовщик,...,[],"[{""companyName"": ""ООО\""АГРОТОРГ\"""", ""dateFrom""...","Сменный-график,Полный-рабочий-день",30000,Полная-занятость,Готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,"{""idUser"": ""2f73dc30-23f5-11e5-b38b-1ff7059456..."
2,47360,162728b0-25dc-11eb-96a2-037acc02728d,c250fab0-25d9-11eb-96a2-037acc02728d,200000000000,200000000000,Башкортостан-республика,NaN,Мужской,NaN,Рабочей,...,[],[],"Гибкий-график,Сменный-график,Полный-рабочий-де...",60000,Полная-занятость,Готов-к-переобучению,Готов-к-командировкам,[],Готов-к-переезду,"{""idUser"": ""ac7e4230-fd77-11e6-8b66-5d9e90ab95..."


In [11]:
#Смотрим на информацию
df_resum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   5000 non-null   int64  
 1   id                           5000 non-null   object 
 2   candidateId                  5000 non-null   object 
 3   stateRegionCode              5000 non-null   int64  
 4   locality                     5000 non-null   int64  
 5   localityName                 4982 non-null   object 
 6   birthday                     4858 non-null   object 
 7   gender                       5000 non-null   object 
 8   age                          4858 non-null   float64
 9   positionName                 5000 non-null   object 
 10  dateCreate                   5000 non-null   object 
 11  dateModify                   5000 non-null   object 
 12  publishedDate                4988 non-null   object 
 13  academicDegree    

In [31]:
df_resum['salary'].min()

0

In [32]:
#Отберем нужные данные
col=['localityName', 'birthday', 'age', 'gender', 'positionName',
     'experience',
      'educationList', 'hardSkills', 'softSkills', 'workExperienceList',
      'scheduleType', 'salary', 'busyType', 'languageKnowledge', 'relocation']

In [14]:
df_resum=df_resum[col]

In [15]:
df_resum.head()

,localityName,birthday,age,gender,positionName,experience,educationList,hardSkills,softSkills,workExperienceList,scheduleType,salary,busyType,languageKnowledge,relocation
0,"Забайкальский-край,-Тунгокоченский-район,-Верш...",1971-05-29T00:00:00+0300,53.0,Мужской,разнорабочий,0,"[{""type"": ""Education""}]",[],[],"[{""companyName"": ""АО ЗабТЭК"", ""dateFrom"": ""202...",Полный-рабочий-день,35000,Полная-занятость,[],Не-готов-к-переезду
1,Татарстан-республика,1988-11-23T00:00:00+0300,35.0,Мужской,Кладовщик,1,"[{""graduateYear"": 2008, ""instituteName"": ""Проф...",[],[],"[{""companyName"": ""ООО\""АГРОТОРГ\"""", ""dateFrom""...","Сменный-график,Полный-рабочий-день",30000,Полная-занятость,[],Не-готов-к-переезду
2,Башкортостан-республика,NaN,NaN,Мужской,Рабочей,0,"[{""graduateYear"": 1991, ""instituteName"": ""Пту ...",[],[],[],"Гибкий-график,Сменный-график,Полный-рабочий-де...",60000,Полная-занятость,[],Готов-к-переезду
3,"Челябинская-область,-г-Челябинск",1990-12-29T00:00:00+0300,33.0,Женский,Продавец,0,"[{""graduateYear"": 2007, ""instituteName"": ""Школ...",[],[],"[{""companyName"": ""ИП Казаков"", ""dateFrom"": ""20...",Неполный-рабочий-день,25000,Полная-занятость,[],Не-готов-к-переезду
4,Свердловская-область,1973-12-14T00:00:00+0300,50.0,Женский,кладовщик-или-продавец,5,"[{""graduateYear"": 1992, ""instituteName"": ""спту...",[],[],"[{""companyName"": ""ооо \""Пульс\"""", ""dateFrom"": ...",Полный-рабочий-день,30000,Полная-занятость,[],Готов-к-переезду


In [16]:
# Сбор из таблицы текстовый файл, если данные изначально в таблице
output_text = ""

# Перебор строк DataFrame
for index, row in df_resum.iterrows():
    row_text = ""
    # Мы через "абракадабру" .qqqrrrqqq\n сохраняем связку с номером
    row_text+=f'\n{index}.qqqrrrqqq\n'
    # Проходим по каждой колонка и плюсуем её к тексту резюме с новой строки \n
    for col in df_resum.columns:
        row_text += f"{col}: {row[col]}\n"
    # В конце добавляем ещё одну "абракадабру" чтобы было понятно что резюме закончилось
    row_text += "\nENDOFR\n\n"
    output_text += row_text

# Запись результата в файл
with open('output.txt', 'w') as file:
    file.write(output_text)

print("Текст успешно записан в файл")

Текст успешно записан в файл


In [17]:
# Смотрим что записалось, выводя первые несколько строк
with open('output.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line, end='')


0.qqqrrrqqq
localityName: Забайкальский-край,-Тунгокоченский-район,-Вершино-Дарасунский-поселок-городского-типа
birthday: 1971-05-29T00:00:00+0300
age: 53.0
gender: Мужской
positionName: разнорабочий
experience: 0
educationList: [{"type": "Education"}]
hardSkills: []
softSkills: []
workExperienceList: [{"companyName": "АО ЗабТЭК", "dateFrom": "2022-02-01T00:00:00+0300", "dateTo": "2022-05-01T00:00:00+0300", "demands": "<p>ремонт котельного оборудования</p>", "jobTitle": "слесарь-ремонтник", "relevant": false, "type": "WorkExperience"}]
scheduleType: Полный-рабочий-день
salary: 35000
busyType: Полная-занятость
languageKnowledge: []
relocation: Не-готов-к-переезду

ENDOFR


1.qqqrrrqqq
localityName: Татарстан-республика
birthday: 1988-11-23T00:00:00+0300
age: 35.0
gender: Мужской
positionName: Кладовщик
experience: 1
educationList: [{"graduateYear": 2008, "instituteName": "Профессиональное училище номер 2", "type": "Education"}]
hardSkills: []
softSkills: []
workExperienceList: [{"compa

In [40]:
def replace_parameters(line):
    replacements = {
        "localityName": "Название Местности",
        "birthday": "Дата Рождения",
        "age": "Возраст",
        "gender": "Пол",
        "positionName": "Название Должности",
        "experience": "Опыт",
        "educationList": "Список Образования",
        "hardSkills": "Профессиональные Навыки",
        "softSkills": "Общие Навыки",
        "workExperienceList": "Список Рабочего Опыта",
        "companyName": "Название Компании",
        "dateFrom": "Дата Начала",
        "dateTo": "Дата Окончания",
        "demands": "Требования",
        "jobTitle": "Название Работы",
        "relevant": "Актуально",
        "type": "Тип",  # Исправлена ошибка с кириллической "т"
        "scheduleType": "Тип Графика",
        "salary": "Зарплата",
        "busyType": "Тип Занятости",
        "languageKnowledge": "Знание Языков",
        "relocation": "Готовность к Переезду"
    }

    for english, russian in replacements.items():
        line = line.replace(english, russian)

    return line  # Исправленный отступ

# Чтение файла, замена параметров и запись в новый файл
with open('output.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

with open('output_translated.txt', 'w', encoding='utf-8') as file:  # Удален лишний пробел в названии
    for line in lines:
        file.write(replace_parameters(line))

print("Файл успешно переведен и сохранен как 'output_translated.txt'")


Файл успешно переведен и сохранен как 'output_translated.txt'


In [43]:
# Смотрим что записалось, выводя первые несколько строк
with open('/content/output_translated.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line, end='')



0.qqqrrrqqq
Название Местности: Забайкальский-край,-Тунгокоченский-район,-Вершино-Дарасунский-поселок-городского-типа
Дата Рождения: 1971-05-29T00:00:00+0300
Возраст: 53.0
Пол: Мужской
Название Должности: разнорабочий
Опыт: 0
Список Образования: [{"Тип": "Education"}]
Профессиональные Навыки: []
Общие Навыки: []
Список Рабочего Опыта: [{"Название Компании": "АО ЗабТЭК", "Дата Начала": "2022-02-01T00:00:00+0300", "Дата Окончания": "2022-05-01T00:00:00+0300", "Требования": "<p>ремонт котельного оборудования</p>", "Название Работы": "слесарь-ремонтник", "Актуально": false, "Тип": "WorkExperience"}]
Тип Графика: Полный-рабочий-день
Зарплата: 35000
Тип Занятости: Полная-занятость
languВозрастKnowledge: []
Готовность к Переезду: Не-готов-к-переезду

ENDOFR


1.qqqrrrqqq
Название Местности: Татарстан-республика
Дата Рождения: 1988-11-23T00:00:00+0300
Возраст: 35.0
Пол: Мужской
Название Должности: Кладовщик
Опыт: 1
Список Образования: [{"graduateYear": 2008, "instituteName": "Профессиональное

Создание базы данных FAISS и поисковика.

In [54]:
file_path = '/content/output.txt'
encoding = 'utf-8'
delimiter = 'ENDOFR'

# Чтение содержимого файла
with open(file_path, 'r', encoding=encoding) as file:
    text = file.read()

# Разделение текста по заданному разделителю
source_chunks = text.split(delimiter)  # Исправлено имя переменной

# Проверка, есть ли хотя бы 3 элемента
if len(source_chunks) > 2:
    print("source_chunks[2]:", source_chunks[3])
else:
    print("В файле меньше 3 частей после разбиения.")


source_chunks[2]: 


3.qqqrrrqqq
localityName: Челябинская-область,-г-Челябинск
birthday: 1990-12-29T00:00:00+0300
age: 33.0
gender: Женский
positionName: Продавец
experience: 0
educationList: [{"graduateYear": 2007, "instituteName": "Школа", "type": "Education"}]
hardSkills: []
softSkills: []
workExperienceList: [{"companyName": "ИП Казаков", "dateFrom": "2019-09-01T00:00:00+0300", "dateTo": "2019-11-01T00:00:00+0300", "demands": "<p>Продажа непродовольственных товаров&nbsp;</p>", "jobTitle": "Продавец", "relevant": false, "type": "WorkExperience"}]
scheduleType: Неполный-рабочий-день
salary: 25000
busyType: Полная-занятость
languageKnowledge: []
relocation: Не-готов-к-переезду




Преобразуем кусочки резюме в объект Document, чтобы работать с ним в рамках FAISS. Берём документы и используем эмбеддинг-модель, выбранную ранее.

In [57]:
# Преобразоание строк в объекты Document (lang_chain)
documents = [Document (page_content=chunk) for chunk in source_chunks]

# Создание базы знаний с использованием FAISS
db = FAISS.from_documents(documents, embedding_model)

print('База знаний создана!')

База знаний создана!


In [64]:
vac = """Вакансия: Инженер-программист

**Компания:** {company}
**Местоположение:** {location}
**Тип занятости:** {employment_type}
**Уровень:** {level}
**Зарплата:** {salary}

#### **Обязанности:**
- Разработка, тестирование и поддержка программного обеспечения.
- Оптимизация и рефакторинг кода для повышения производительности.
- Разработка архитектуры приложений и участие в проектировании решений.
- Работа с базами данных (создание, оптимизация запросов, администрирование).
- Взаимодействие с командой разработки, аналитиками и тестировщиками.
- Поддержка и развитие существующих проектов.
- Документирование кода и технических решений.

#### **Требования:**
- Опыт работы в разработке программного обеспечения {experience} лет.
- Знание языков программирования: {languages}.
- Опыт работы с фреймворками: {frameworks}.
- Знание реляционных ({sql_dbs}) и NoSQL ({nosql_dbs}) баз данных.
- Опыт работы с системами контроля версий ({version_control}).
- Понимание принципов ООП, алгоритмов и структур данных.
- Опыт работы с облачными сервисами ({cloud_services}) будет плюсом.
- Опыт работы с контейнерами ({containers}) будет преимуществом.
- Умение работать в команде, аналитический склад ума, ответственность.

#### **Будет плюсом:**
- Опыт работы с CI/CD ({ci_cd}).
- Навыки DevOps и автоматизации.
- Опыт работы с микросервисной архитектурой.

#### **Условия:**
- Конкурентная зарплата и система бонусов.
- Возможность удалённой работы / гибкий график.
- Работа над интересными и сложными проектами.
- Дружный коллектив и поддержка команды.
- Возможность обучения и профессионального роста.

Если вас заинтересовала вакансия, отправьте резюме на {email} или свяжитесь с нами по телефону {phone}.
"""

# Пример заполнения значений
vacancy_text = vac.format(
    company="ООО 'TechCorp'",
    location="Удаленно",
    employment_type="Контракт",
    level="Junior",
    salary="150000-200000 руб",
    experience="2-3",
    languages="Python, Java, C++, JavaScript",
    frameworks="Django, Flask, Spring, React",
    sql_dbs="PostgreSQL, MySQL",
    nosql_dbs="MongoDB, Redis",
    version_control="Git",
    cloud_services="AWS, Google Cloud, Azure",
    containers="Docker, Kubernetes",
    ci_cd="Jenkins, GitLab CI/CD",
    email="email@example.com",
    phone="+7 (900) 123-45-67"
)

print(vacancy_text)


Вакансия: Инженер-программист  

**Компания:** ООО 'TechCorp'  
**Местоположение:** Удаленно  
**Тип занятости:** Контракт  
**Уровень:** Junior  
**Зарплата:** 150000-200000 руб  

#### **Обязанности:**  
- Разработка, тестирование и поддержка программного обеспечения.  
- Оптимизация и рефакторинг кода для повышения производительности.  
- Разработка архитектуры приложений и участие в проектировании решений.  
- Работа с базами данных (создание, оптимизация запросов, администрирование).  
- Взаимодействие с командой разработки, аналитиками и тестировщиками.  
- Поддержка и развитие существующих проектов.  
- Документирование кода и технических решений.  

#### **Требования:**  
- Опыт работы в разработке программного обеспечения 2-3 лет.  
- Знание языков программирования: Python, Java, C++, JavaScript.  
- Опыт работы с фреймворками: Django, Flask, Spring, React.  
- Знание реляционных (PostgreSQL, MySQL) и NoSQL (MongoDB, Redis) баз данных.  
- Опыт работы с системами контроля верс

Обращаемся к базе данных. key_words — значение, по которому мы ищем, k — количество строк, которое должно быть выдано.

In [67]:
key_words = vac  # Убедитесь, что vac — это строка с текстом вакансии
k = 5

# Поиск схожих документов
sim_docs = db.similarity_search(key_words, k=k)

# Вывод результатов
for i, doc in enumerate(sim_docs):
    print(f"Документ {i+1}: {doc.page_content}\n")


Документ 1: 


4766.qqqrrrqqq
localityName: Краснодарский-край,-г-Сочи
birthday: 1976-02-24T00:00:00+0300
age: 48.0
gender: Мужской
positionName: руководитель-подразделения,-ведущий-специалист
experience: 3
educationList: [{"faculty": "радиотехнический", "graduateYear": 1999, "instituteName": "«Томский государственный университет систем управления и радиоэлектроники»", "qualification": "инженер по специальности", "speciality": "инженер", "type": "Education"}]
hardSkills: []
softSkills: []
workExperienceList: [{"achievements": "<p><strong>Профессиональные навыки и компетенции:</strong> • Опыт управления персоналом более 5 лет</p>\n<ul>\n<li>Наличие понимания и практического опыта ведения бизнеса в Интернет</li>\n<li>Навыки проведения переговоров на уровне первых лиц компании</li>\n<li>Опыт в получении лицензий и разрешений от контролирующих органов</li>\n<li>Опыт в разработке различных положений о порядке использования IT ресурсов компании, должностных инструкций, регламентов взаимодейс

Добавляем код, который сделает выдачу более красивой. Это важно на этапе продакшена. Нам нужна “белиберда” (r'(\d+)\.qqqrrrqqq'), чтобы обратиться к полному тексту резюме. Это создаёт связь между поиском и выдачей. Также она полезна для подсчёта количества резюме внутри — например, нужно выбрать подвыборку из сотни резюме. Возвращаем порядковый номер и содержание резюме. По индексу мы обращаемся к исходной базе данных.

In [82]:
key_words="менеджер"
k=5
sim_docs=db.similarity_search (key_words, k=k)

# функция для извлечения номера строки из текста
def extract_index(text):
	# Используем регулярное выражение для поиска цифр перед ".qqqrrrqqq"
	# Это полезно когда ищешь не по всей вакансии, а по сущностным моментам!
	match = re.search (r'(\d+)\.qqqrrrqqq', text)
	if match:
		return match.group(1) # Возвращаем найденное число
	return None # Если ничего не найдено, возвращаем None

print ('Найденные', k, 'фрагментов резюме: \n')

for num, i in enumerate(sim_docs):
    ind = extract_index(str(i))
    print('Вакансия:', num + 1)
    print('_____________________________')
    display(df_resum.loc[int(ind)])
    print(i.page_content)
    print()
    print()


Найденные 5 фрагментов резюме: 

Вакансия: 1
_____________________________


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'